In [1]:
from allennlp.predictors.predictor import Predictor as AllenNLPPredictor

class PythonPredictor:
    def __init__(self, config=None):
        self.predictor = AllenNLPPredictor.from_path(
            "bidaf-elmo-model-2018.11.30-charpad.tar"
        )

    def predict(self, payload):
        """
        :param payload: dict containing the keys "passage" and "question" - both keys point to string values. 
        "passage" refers to the source doc that the model will look at while "question" refers to the question 
        asked to the model.
        :returns: a string representing the most probable answer, according to the model.
        """
        prediction = self.predictor.predict(
            passage=payload["passage"], question=payload["question"]
        )
        return prediction["best_span_str"]
    
    def full_predict(self, payload):
        """
        :param payload: dict containing the keys "passage" and "question" - both keys point to string values. 
        "passage" refers to the source doc that the model will look at while "question" refers to the question 
        asked to the model.
        :returns: a dict representing the predictions made by model.
        """
        prediction = self.predictor.predict(
            passage=payload["passage"], question=payload["question"]
        )
        return prediction

In [2]:
predictor = PythonPredictor()
type(predictor.predictor)

_jsonnet not loaded, treating C:\Users\melvin\AppData\Local\Temp\tmp1jo963t0\config.json as json
_jsonnet not loaded, treating snippet as json
D:\melvin\proj-minecraft\env\lib\site-packages\torch\nn\modules\rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
D:\melvin\proj-minecraft\env\lib\site-packages\allennlp\data\token_indexers\token_characters_indexer.py:56: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


allennlp.predictors.bidaf.BidafPredictor

In [3]:
# example prediction
payload = {
    "passage": "The trial judges accepted that both the appellants had come into Singapore only with a view to boarding a flight to Amsterdam the next day. They, however, rejected the submission made on behalf of the appellants that bringing drugs into Singapore with a view solely of exporting them would not be an offence under s 7 of the Act. They also rejected Ko’s defence that he did not know that what he was carrying was diamorphine. Accordingly, they convicted the appellants. Against the convictions, this appeal was brought. At the conclusion we dismissed it, and we now give our reasons.Ground (a) can be disposed of very briefly. By s 18(2) of the Act a rebuttable presumption arose that Ko knew the nature of the drug that he was carrying. Once the presumption arose, the onus of discharging it was on Ko. Having heard Ko’s defence, the trial judges were satisfied that he had not discharged the presumption. We have reviewed the record and it is clear that the trial judges were entitled on the evidence before them to arrive at this finding. We saw no reason to interfere.and submitted that s 7 was applicable only when it was sought to punish a master or captain who had contravened s 20. We could not accept that submission. In common with a number of other similar provisions in the Act, what s 20 does is to raise a presumption as to knowledge. By s 20, if it is proved that a drug was found in a ship or aircraft, then the presumption would arise that the drug was imported in the ship or aircraft with the knowledge of the master or captain. No doubt, in such a case, a master or captain may be charged for violating s 7 of the Act but that does not mean to say that s 7 is confined in its operations only to the master of a ship or captain of an aircraft used for the import of drugs. We see no reason why s 7 should not operate against (say) a passenger in a ship or aircraft who was importing drugs. Against such a passenger the presumption under s 20 as to knowledge would obviously not be applicable but (as in this case) the presumption under s 18(2) would apply.",
    "question": "was the appeal dismissed?"
}
prediction = predictor.full_predict(payload)
print(prediction.keys())
prediction['best_span_str']

dict_keys(['passage_question_attention', 'span_start_logits', 'span_start_probs', 'span_end_logits', 'span_end_probs', 'best_span', 'best_span_str', 'question_tokens', 'passage_tokens'])


'rejected Ko’s defence that he did not know that what he was carrying was diamorphine. Accordingly, they convicted the appellants. Against the convictions, this appeal was brought. At the conclusion we dismissed it'

## From extracted text (in json) to answers
1. present whole document and query to system.
2. search for potentially relevant paragraphs; this narrows the search space for the subsequent QnA NLP model. 
2. from these paragraphs, search for answer.
3. print top answer (or top few answers)

In [4]:
import re

class primitiveSearchEngine:
    def __init__(self):
        pass
        
    def and_search(self, itr, queries):
        """
        Searches for the passages/paragraphs that contain a 
        co-occurence of the exact query terms, in any order.
        
        :params itr: a dict containing strings to search through.
        :params queries: a list of query terms.
        :returns: a dict of the form, {key: search_result}.
        """
        regex = "^"
        for term in queries:
            # regex = regex + term + '|'
            regex = regex + rf"(?=.*\b{term}\b)"
        regex = regex + ".*$"
        
        # note: this regex pattern searches for the co-occurence of the
        # exact specified terms, in any order.
        
        pattern = re.compile(regex)
        
        results = {}
        
        for k, v  in itr.items():
            match = pattern.search(v)
            
            if match:
                results[k] = v
        return results
    
    def or_search(self, itr, queries):
        """
        Searches for the paragraphs/strings that contain any of the query terms.
        
        :params itr: a dict containing strings to search through. they key can be a para number.
        :params queries: a list of query terms.
        :returns: a dict of the form, {key: search_result}.
        """
        results = {}
        
        for k, v  in itr.items():
            for term in queries:
                if term in v:
                    results[k] = v
            
        return results
    
    def rule1(self, itr, queries, scorethreshold):
        """
        Rule 1 is an OR search and gives an equal weightage to each keyword
        
        :param scorethreshold: integer. min number of relevant terms that must appear in 
        a text (could be a paragaph). 
        :returns: a dictionary. keys are a subset of itr.keys() and ea value is rule1's 
        relevance score.
        """
        output = dict()
        for para in itr:
            score = 0
            for word in queries:
                if word in itr[para]:
                    score += 1 
            if score >= scorethreshold: 
                output[para] = score

        output = {k: v for k, v in sorted(output.items(), key=lambda x: x[1], reverse=True)}
        return output

In [5]:
# Porter stemming
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

def tokenise(string): # works on any arbitrary string
    tokens = []
    for sentence in sent_tokenize(string):
        for token in word_tokenize(sentence):
            tokens.append(token)
    return tokens

def stem(token): # tokenizes any particular token
    return PorterStemmer().stem(token)

## Run predictions on documents with 1 simple type of question.

In [7]:
'''
1. pull out the paragraphs of a valid document
2. run search engine to find relevant paragraphs
3. concatenate relevant paragraphs into a new "source" document
4. feed source and question into QnA model, get answer.
5. manually evaluate if the answer is able to answer the question.
6. auto evaluate if the predicted answer is the same or overlaps with the ground-truth answer.

7. repeat 1-6 for every doc and every question we want to test.

a. we start with questions about whether the accused was convicted. find one good phrasing.
b. can consider testing other phrasings for the same question as well.
c. then we identify other high-prio questions to test.
d. tabulate model's accuracy in answering each type of question.
'''

'\n1. pull out the paragraphs of a valid document\n2. run search engine to find relevant paragraphs\n3. concatenate relevant paragraphs into a new "source" document\n4. feed source and question into QnA model, get answer.\n5. manually evaluate if the answer is able to answer the question.\n6. auto evaluate if the predicted answer is the same or overlaps with the ground-truth answer.\n\n7. repeat 1-6 for every doc and every question we want to test.\n\na. we start with questions about whether the accused was convicted. find one good phrasing.\nb. can consider testing other phrasings for the same question as well.\nc. then we identify other high-prio questions to test.\nd. tabulate model\'s accuracy in answering each type of question.\n'

In [6]:
import pandas as pd
import numpy as np
import json
# with open('data/cases.json') as f:
#     cases = json.load(f)

In [10]:
'''
Set up the output_df dataframe from the .json output file.
(based on glowsplint's test_output.py code)
'''
output_df = pd.read_json('output.json')

court_map = {
    'High Court': 'SGHC',
    'Court of Appeal': 'SGCA',
    'District Court': 'SGDC',
    'Court of Criminal Appeal': 'SGCA'
}

output_df['Court'] = output_df.Court.map(court_map)
ref_columns = {
    0: 'refSLR',
    1: 'ref'
}

# Split reference into SLR reference and normal reference
output_df[list(ref_columns.values())] = pd.DataFrame(
    output_df.reference.apply(pd.Series)).rename(columns=ref_columns)
ref_na = output_df['ref'].isna()
output_df.loc[ref_na, 'ref'], output_df.loc[ref_na,'refSLR'] = output_df.refSLR[ref_na], np.nan
output_df.drop('reference', axis=1, inplace=True)

# retain the ordered list of parties in one column, before exploding output_df['parties'] next.
output_df = output_df.assign(listed_parties=output_df.parties)

# Construct the unique reference
exploded_output = output_df.explode('parties')
output_df = exploded_output.loc[exploded_output.parties.str.lower(
) != "public prosecutor"].copy()
output_df['unique_ref'] = output_df['ref'] + \
    ' ' + output_df['parties']
output_df.unique_ref = output_df.unique_ref.str.upper().str.replace(
    "AND ANOTHER", "").str.replace("AND OTHERS", "").str.replace(' +', ' ').str.strip()
output_df = output_df[['unique_ref', 'case_id', 'date', 'Court',
                                 'coram', 'counsel', 'listed_parties', 'parties', 'paragraphs']]
output_df.rename(columns={'parties': 'accused'}, inplace=True)

In [11]:
output_df.head()

,unique_ref,case_id,date,Court,coram,counsel,listed_parties,accused,paragraphs
0,[1989] SGHC 75 GOH AH LIM,Criminal Case No 6 of 1988,1989-08-24,SGHC,"[Lai Kew Chai J, F A Chua J]","{'prosecution': ['Lee Sing Lit'], 'defence': [...","[Public Prosecutor, Goh Ah Lim]",Goh Ah Lim,"{'1': 'The accused, a male Chinese aged 46, fa..."
1,[1989] SGHC 9 KADIR BIN AWANG,Criminal Case No 2 of 1988,1989-02-03,SGHC,"[T S Sinnathuray J, Joseph Grimberg JC]","{'prosecution': ['Lee Sing Lit'], 'defence': [...","[Public Prosecutor, Kadir bin Awang]",Kadir bin Awang,{'1': 'Kadir bin Awang (“the accused”) was cha...
2,[1990] SGHC 18 KO MUN CHEUNG,Criminal Case No 17 of 1988,1990-03-15,SGHC,"[Chan Sek Keong J, Yong Pung How J]","{'prosecution': ['Seng Kwang Boon'], 'defence'...","[Public Prosecutor, Ko Mun Cheung and another]",Ko Mun Cheung and another,"{'1': 'You, Ko Mun Cheung, Raymond, (“Ko”) are..."
3,[1991] SGCA 14 SIM AH CHEOH,Criminal Appeal No 12 of 1988,1991-05-31,SGCA,"[Yong Pung How CJ, Chan Sek Keong J, L P Thean J]","{'prosecution': ['Chan Seng Onn'], 'defence': ...","[Sim Ah Cheoh and others, Public Prosecutor]",Sim Ah Cheoh and others,"{'1': 'The first appellant, Sim Ah Cheoh (“Sim..."
4,[1991] SGHC 147 NG CHONG TECK,Criminal Case No 63 of 1990,1991-10-12,SGHC,"[P Coomaraswamy J, Kan Ting Chiu JC]","{'prosecution': ['Ong Hian Sun'], 'defence': [...","[Public Prosecutor, Ng Chong Teck]",Ng Chong Teck,{'1': 'The accused was tried before us and con...


In [13]:
# filter out cases that are definitely trial cases
trials_indices = []
for i in range(len(output_df)):
    if "criminal case" in output_df.iloc[i]["case_id"].lower():
        trials_indices.append(i)
    elif output_df.iloc[i]["Court"] == 'SGDC':
        trials_indices.append(i)

cases_df = output_df.iloc[trials_indices]
cases_df.head()

,unique_ref,case_id,date,Court,coram,counsel,listed_parties,accused,paragraphs
0,[1989] SGHC 75 GOH AH LIM,Criminal Case No 6 of 1988,1989-08-24,SGHC,"[Lai Kew Chai J, F A Chua J]","{'prosecution': ['Lee Sing Lit'], 'defence': [...","[Public Prosecutor, Goh Ah Lim]",Goh Ah Lim,"{'1': 'The accused, a male Chinese aged 46, fa..."
1,[1989] SGHC 9 KADIR BIN AWANG,Criminal Case No 2 of 1988,1989-02-03,SGHC,"[T S Sinnathuray J, Joseph Grimberg JC]","{'prosecution': ['Lee Sing Lit'], 'defence': [...","[Public Prosecutor, Kadir bin Awang]",Kadir bin Awang,{'1': 'Kadir bin Awang (“the accused”) was cha...
2,[1990] SGHC 18 KO MUN CHEUNG,Criminal Case No 17 of 1988,1990-03-15,SGHC,"[Chan Sek Keong J, Yong Pung How J]","{'prosecution': ['Seng Kwang Boon'], 'defence'...","[Public Prosecutor, Ko Mun Cheung and another]",Ko Mun Cheung and another,"{'1': 'You, Ko Mun Cheung, Raymond, (“Ko”) are..."
4,[1991] SGHC 147 NG CHONG TECK,Criminal Case No 63 of 1990,1991-10-12,SGHC,"[P Coomaraswamy J, Kan Ting Chiu JC]","{'prosecution': ['Ong Hian Sun'], 'defence': [...","[Public Prosecutor, Ng Chong Teck]",Ng Chong Teck,{'1': 'The accused was tried before us and con...
7,[1992] SGHC 17 NG KWOK CHUN,Criminal Case No 60 of 1990,1992-01-31,SGHC,"[S Rajendran J, MPH Rubin JC]","{'prosecution': ['Ong Hian Sun'], 'defence': [...","[Public Prosecutor, Ng Kwok Chun and another]",Ng Kwok Chun and another,"{'1': 'Ng Kwok Chun (“Ng”), 27 years of age, a..."


In [15]:
len(cases_df)

207

In [16]:
searchEngine = primitiveSearchEngine()

ConvictionTrialQ = ["accordingly", "acquit", "charge", "convict", "element", "guilty", "made out", "prove", "reasonable doubt", "reasons", "satisfied", "sentence", "therefore" ]
# ConvictionAppealQ = ConvictionTrialQ + ["affirm", "allow", "dismiss"]

#add porter stemming to capture more relevant words.
search_terms = ConvictionTrialQ
search_terms = [stem(term) for term in search_terms] # remove if stemming doesn't help

In [26]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
def predict_multiple(cases_df, search_terms, print_output=True):
    """
    todo: put under the predictor class.
    """
    preds = []
    sentences = []
    for i in range(len(cases_df)):
        print(f"case {i+1}/{len(cases_df)}")

        accused = cases_df.iloc[i]["accused"]
        and_index = accused.find("and")
        if and_index != -1:
            # if 'NAME and others' then cut away ' and others'
            accused = accused[0:and_index - 1]

        qn = f"was {accused} found guilty?"
        #qn = f"was {appellant}'s appeal allowed?"
        # todo: modify this code chunk to ask multiple question-phrasings at once
        # instead of only one question-phrasing per document.
        
        # Find paragraphs related to search terms; can also try finding sentences instead, later on.
        # rule1() yields item numbers (e.g. paragraph numbers) that contain ANY of the search terms. 
        SCORE_THRESHOLD = 2  # arbitrary threshold for "rule1"; result must be greater than thres.
        results = searchEngine.rule1(cases_df.iloc[i]['paragraphs'], search_terms, SCORE_THRESHOLD)
        
        # Concatenate top relevant paragraphs into one chunk to feed as input to the QnA model.
        combined_psg = ""
        total_score = 0
        MAX_PARAS = 4
        num_paras = 0
        relevant_paras = list()
        if len(results) > 0:    
            keys_list = list(results.keys())
            max_score = results[keys_list[0]]  # because keys_list is presorted according to
            # descending score
            for item_num, score in results.items():               
                if score == max_score:
                    combined_psg = combined_psg + " " + cases_df.iloc[i]['paragraphs'][str(item_num)]
                    total_score = total_score + score
                    num_paras = num_paras + 1
                    relevant_paras.append(item_num)
                    total_score = total_score + score
                elif num_paras < MAX_PARAS:
                    combined_psg = combined_psg + " " + cases_df.iloc[i]['paragraphs'][str(item_num)]
                    num_paras = num_paras + 1
                    relevant_paras.append(item_num)
                    total_score = total_score + score

        sentence_str = 'NA' # prediction defaults to NA if no relevant paras found.
        if combined_psg:
            payload = {
                'passage': combined_psg,
                'question': qn
            }
            prediction = predictor.full_predict(payload)
            predicted_span = prediction['best_span_str']
            span_indices = prediction['best_span']
            
            tokens = prediction['passage_tokens']
            start_index = span_indices[0]
            end_index = span_indices[1]
            
            sent_start_index = start_index
            while tokens[sent_start_index] != '.' and sent_start_index > 0:
                sent_start_index = sent_start_index - 1
            sent_end_index = end_index
            while sent_end_index < len(tokens):
                if tokens[sent_end_index] != '.':
                    sent_end_index = sent_end_index + 1
                else:
                    break
            
            if tokens[sent_start_index] == '.':
                sentence = tokens[sent_start_index + 1 : sent_end_index]
            else:
                sentence = tokens[sent_start_index : sent_end_index]
            detokenizer = TreebankWordDetokenizer()
            sentence_str = detokenizer.detokenize(sentence)  # the output is not perfectly formatted.
            
        preds.append(predicted_span)
        sentences.append(sentence_str)
        
        if print_output:
            # TODO: use logger.
            print(cases_df.iloc[i]["unique_ref"])
            print(f"qn: {qn}")
            print("relevant paragraphs:\n" + combined_psg)
            print(f"relevant paragraph numbers: {relevant_paras}")
            print(f"keywords score: {total_score}.")
            print()
            print(f"predicted span: {predicted_span}")
            print(f"full sentence or sequence: {sentence_str}")
            #print(f"item reference number: {answer_item_num}")  # determining this isn't straightforward.
            #print(f"start and end indices of tokens in passage: {span_indices}")
            print()
    return preds, sentences

In [27]:
preds, sentences = predict_multiple(cases_df, search_terms)

case 1/207
[1989] SGHC 75 GOH AH LIM
qn: was Goh Ah Lim found guilty?
relevant paragraphs:
 At the close of the case for the Prosecution, the Defence did not make any submission. We were of the view that Prosecution had made out a prima facie case which, if unrebutted, would warrant a conviction of the charges in the alternative and we called upon the accused to enter upon his defence. We administered the usual allocution. The accused elected to give evidence on oath. Accordingly, we convicted the accused of the offence of attempting to export to Australia the diamorphine in the quantity as charged contrary to s 7 read with s 12 of the Act. We stood down the alternative charge. We sentenced him to death. Sgt John Cheong Ging Thow arrived at Holding Room D63 at about 5.25pm. He was briefed by the police officers. He saw the accused and approached him. He checked his waist band and touched it. It was soft. He then recited his questioning and the answers of the accused. Counsel did not ch

In [35]:
# display length of each predicted answer in terms of no. of tokens.
from statistics import median

tokenized_answers = list()
for sent in sentences:
    tokens = word_tokenize(sent)
    tokenized_answers.append(tokens)

print("lengths of answers:")
lengths = [len(ans) for ans in tokenized_answers]
print(lengths)
print(f"min: {min(lengths)}")
print(f"max: {max(lengths)}")
print(f"avg: {sum(lengths)/len(lengths)}")
print(f"median: {median(lengths)}")

lengths of answers:
[40, 86, 13, 53, 52, 40, 20, 54, 35, 16, 15, 35, 14, 13, 34, 34, 76, 29, 20, 41, 37, 33, 20, 16, 32, 32, 25, 26, 51, 52, 33, 13, 84, 13, 34, 13, 18, 14, 23, 34, 37, 19, 63, 8, 36, 32, 21, 11, 1420, 14, 37, 60, 586, 16, 39, 15, 33, 9, 35, 107, 15, 24, 127, 37, 103, 9, 8, 82, 41, 96, 27, 37, 14, 105, 11, 16, 13, 21, 21, 13, 13, 34, 75, 13, 44, 14, 39, 94, 9, 90, 15, 15, 15, 15, 15, 41, 10, 13, 27, 27, 27, 24, 12, 14, 14, 142, 17, 49, 7, 12, 13, 10, 32, 20, 15, 48, 92, 19, 19, 61, 24, 31, 20, 20, 43, 11, 11, 8, 36, 10, 288, 288, 14, 16, 21, 20, 48, 24, 20, 48, 15, 123, 61, 71, 10, 10, 30, 98, 174, 174, 10, 10, 80, 23, 9, 9, 8, 10, 18, 15, 12, 24, 86, 25, 39, 77, 12, 13, 17, 306, 306, 4, 10, 137, 137, 8, 28, 119, 261, 35, 261, 28, 43, 43, 9, 27, 24, 24, 138, 27, 13, 21, 21, 489, 16, 33, 33, 386, 41, 32, 32, 50, 149, 50, 19, 19, 10]
min: 4
max: 1420
avg: 56.32367149758454
median: 26


In [34]:
path = input('enter path to save csv file:\n')
to_save = pd.DataFrame()
to_save.insert(0, "unique_ref", cases_df['unique_ref'].values)
to_save.insert(1, "predicted_answer", preds)
to_save.insert(2, "containing_sentences", sentences)
to_save.to_csv(path)

enter path to save csv file:
 data/trials_00.csv
